In [ ]:
import fibsem_tools as fst
import numpy as np
import skimage
import yaml
from pathlib import Path
from enum import Enum

In [ ]:
class Annotation(Enum):
    UNKNOWN = 255
    PRESENT = 1
    ABSENT = 0


In [ ]:
data_config = open("../all_data.yaml")
datasets = yaml.safe_load(data_config)
okays = []
not_okays = []
for dataset, ds_info in datasets["datasets"].items():
    for crop in ds_info["crops"]:
        for c in crop.split(","):
            group = fst.read(Path(datasets["gt_path"]) / dataset / "groundtruth.zarr" / c)
            for label in group:
                if label != "all":
                    for lvl in group[label]:
                        if lvl == "s0":
                            attrs_complements = group[f"{label}/{lvl}"].attrs["cellmap"]["annotation"]["complement_counts"]
                            hist = dict(zip(*np.unique(group[f"{label}/{lvl}"], return_counts=True)))
                            complements = {Annotation(k).name.lower(): v for k, v in hist.items() if k != Annotation.PRESENT.value and v != 0}
                            if complements != attrs_complements:
                                print(Path(datasets["gt_path"])/dataset/"groundtruth.zarr"/c/label/lvl)
                                print(complements, attrs_complements)
                                not_okays.append(Path(datasets["gt_path"])/dataset/"groundtruth.zarr"/c/label/lvl)
                            else:
                                okays.append(Path(datasets["gt_path"])/dataset/"groundtruth.zarr"/c/label/lvl)

In [ ]:
not_okays

In [ ]:
for lblpath in not_okays:
    zarray = fst.access(lblpath, "a")
    attrs_complements = zarray.attrs["cellmap"]["annotation"]["complement_counts"]
    hist = dict(zip(*np.unique(zarray, return_counts=True)))
    complements = {Annotation(k).name.lower(): v for k, v in hist.items() if k != Annotation.PRESENT.value and v != 0}
    if complements != attrs_complements:
        print(zarray.attrs["cellmap"]["annotation"]["complement_counts"])
        print(complements)
        attrs = zarray.attrs.asdict()
        attrs["cellmap"]["annotation"]["complement_counts"] = complements
        zarray.attrs["cellmap"] = attrs["cellmap"]
    else:
        print("oh")